# Introduction to Signal Processing Using Deep Learning

There are many resources for learning how to use Deep Learning to process imagery. However, very few resources exist to demonstrate how to process data from other sensors such as acoustic, siesmic, radio, or radar. In this tutorial, we will introduce some basic methods for utilizing a Convolutional Neural Network (CNN) to process Radio Frequency (RF) signals. 

---
Before we begin, let's verify [WebSockets](http://en.wikipedia.org/wiki/WebSocket) are working on your system.  To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above.  If all goes well, you should see some output returned below the grey cell.  

You will know the lab is processing when you see a solid circle in the top-right of the window that looks like this: ![](jupyter_executing.png)
Otherwise, when it is idle, you will see the following: ![](jupyter_idle.png)
For troubleshooting, please see [Self-paced Lab Troubleshooting FAQ](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) to debug the issue.

In [1]:
two_pi = 2*3.14159

### Introduction to signal detection
When monitoring radio frequency (RF) signals, or similar signals from sensors such as biomedical, temperature, etc., we are often interested in detecting certain signal identifying features. This can become a challenging problem when the signal-of-interest is degraded by noise. Traditional signal detection methods use a range of techniques such as energy detection, “matched filtering”, or other correlation-based processing techniques. Short-duration radio frequency (RF) events can be especially challenging to detect, since the useful data length is limited and long integration times are not possible. Weak signals that are short in duration are some of the most difficult to detect. We will walk you through a simple approach using a Convolutional Neural Network (CNN) to tackle the traditional signal processing problem of detecting signals in noise.

### A little background information
Signal detection theory often assumes that a signal is corrupted with additive white Gaussian noise (AWGN). This type of noise is common in the real-world, and the assumption makes analysis easier. In general, the detection of a signal in noise depends on the signal duration, amplitude, and the corresponding noise process. This becomes more difficult if interfering signals occupy the same frequencies as the signal you wish to detect. This is called co-channel interference. Also, many real-world signals have frequency components that change with time, making signal detection difficult. We will look at one of these types of signals - Linear Frequency-Modulated (LFM) signals. In a follow-on tutorial, we will explore Frequency-Hopped (FH) signals and multi-signal detection scenarios. 

In this tutorial, we will utilize spectrograms, which are 2D representations similar to an image, computed from simulated Radio Frequency (RF) data. Transforming the data into the frequency vs. time domain (i.e., spectrogram), allows us to visualize the energy of a signal over some pre-determined time duration and frequency bandwidth. 

### Linear Frequency-Modulated Signals
A linear frequency-modulated (LFM) signal, or "chirp" signal, ramps up or down in frequency over some time duration. Its center frequency changes with time based on its chirp rate. Chirps are used in many different systems for frequency response measurements and timing. RADAR systems use chirp signals due to the inherent large time-bandwith product available with coherent processing. Another common use is for automatic room equalization in home theater receivers, since chirps can excite a large frequency swath quickly. Chirps can also be used as pilot signals to denote the start of an incoming transmission.

Figure 1 shows a high-SNR chirp as seen in a grayscale spectrogram. Since the spectrogram consists of real numbers, all > 0, we can map it to a 2D image by scaling the values appropriately. In Figure 1, the x axis is time and the y axis is frequency. Brightness is proportional to signal power.

<img src="images2/figure2.png" alt="Drawing" style="width: 400px;"/>
<div align="center">Fig1. High-SNR chirp spectrogram (grayscale).</div>

The chirp in Figure 1 has a high SNR and is easy to detect with traditional signal processing algorithms. But when the environment contains other co-channel interference and noise, reliable detection becomes more difficult. Figure 2 shows an example spectrogram with some pulsed carrier waves (i.e., sinusoids) and a low-bitrate digital communication BPSK signal embedded in noise.

<img src="images2/figure3.png" alt="Drawing" style="width: 400px;"/>
<div align="center">Fig2. Multiple signals and noise (x-axis is time, y-axis is frequency).</div>

In Figure 2 there is no chirp signal, just noise and other signals. However, this is an example of what “real-world” RF signals look like – i.e., combinations of signal types with different strengths, all embedded in noise. Figure 3 consists of another spectrogram showing noise, interfering signals, and a weak chirp signal.

<img src="images2/figure4.png" alt="Drawing" style="width: 400px;"/>
<div align="center">Fig3. Weak chirp embedded in noise with other signals (x-axis is time, y-axis is frequency).</div>

In Figure 3, the chirp signal is several dB below the noise power. In fact, the signal-to-noise-ratio (SNR) for the chirp is -7.0 dB. Note that it is barely visible to the human eye. Traditional detection methods, without large amounts of integration and/or a priori signal information, fail consistently in detecting a weak signal like this. Moreover, since we have interfering signals that are sharing the same bandwidth as the chirp, the problem becomes even harder.

### What is this good for?
When monitoring RF signals, we want accurate detection of signals without the burden of having a human visually inspect all the data. For example, in the case of intelligent spectral monitoring or cognitive radio, we want something to autonomously analyze extraordinary amounts of signal data. The question arises: Can we design a process using deep learning to help detect these weak signals?

# Deep Spectral Detection: Data and Network Creation
We will create a two-output convolutional neural network that ingests a time-frequency spectrogram. The network will determine whether a chirp signal is present (class 0 - signal) or a chirp signal is NOT present (class 1 - noise). We have provided a dataset (i.e., RF Spectrograms) to use for training and validation.

### Starting up DIGITS
We have a pre-configured DIGITS instance set up for use with this tutorial. Click [`HERE`](/digits/) to start up an instance.

The DIGITS server page should be active. Now we will create and ingest the dataset using the tools in DIGITS.

### Creating DIGITS spectrogram database
- Select the Datasets menu. Click the 'Images' button underneath the 'New Dataset' label and choose the 'Classification' menu option. Enter your username.

<img src="images2/Digits_Screenshot1.png" alt="Drawing" style="width: 700px;"/>

- For the Image Type select Grayscale. Keep the image size 256x256 and enter the directory `/data/traindemo` for the training image URL. The other settings used are shown in the picture below.

<img src="images2/Digits_Screenshot2.png" alt="Drawing" style="width: 700px;"/>

- Name the dataset and click 'Create'. The data generation screen will appear showing your progress and job information.

<img src="images2/Digits_Screenshot3.png" alt="Drawing" style="width: 700px;"/>

- After the  data generation process has finished, click the "DIGITS" logo in the upper left webpage menu. This will take you back to the DIGITS main screen. If you select the Datasets menu, you should see your new dataset listed. If you select your new dataset, you will open a page with your dataset information - number of classes, mean spectrogram image it computed (all black), and links to explore spectrogram images within the database. Make sure you click on the button to display the data and explore the spectrograms.

<img src="images2/Digits_Screenshot4.png" alt="Drawing" style="width: 700px;"/>

### New CNN model creation
- Return to the DIGITS main page. While in the Models menu, click the 'Images' button on the right side of the page underneath New Model and select the 'Classification' menu option.

<img src="images2/Digits_Screenshot5.png" alt="Drawing" style="width: 700px;"/>

- In the New Image Classification Model screen, select your new dataset in the upper left box.
- Lower down the page select 'Custom Network' and paste the contents of the following [chirp CNN prototxt](http://datasets.kickview.com:8080/dsd_demo/train_val_digits4_chirp.prototxt) into the window.
    - Select Nesterov in the solver type menu and set the learning rate (LR) to 0.001.
    - Also click on the Advanced button in the learning tab and select Exponential Decay with a Gamma of 0.96. 
    - Note we set the validation epoch to 1 so the network validation will be run once every epoch.
    - Enter 5 for the number of Training Epochs.
    
<img src="images2/Digits_Screenshot6.png" alt="Drawing" style="width: 700px;"/>

- You can also click on the 'Visualize' button next to the custom network window. This will show you the network block diagram.
    - You may notice that we started with an AlexNet and pruned the number of fully-connected layers to 2.
    - The two fully connected layers were also reduced in size (fewer neurons).
    - We added regularization to improve training.
    
<img src="images2/Vis_Screenshot.png" alt="Drawing" style="width: 400px;"/>

- Finally, name the model and click 'Create'. The training screen will appear with status and real-time training performance plots.
    - After training is complete, you can see the network was unable to learn - i.e., the loss curve never decreased.
    - This is an example of how selecting the right learning rate is part of the hyperparameter search space that needs to be explored in order to get a model to train well. Next, we will make changes and train again.
    
<img src="images2/Digits_Screenshot7.png" alt="Drawing" style="width: 700px;"/>

- Click the 'Clone job' button at the top right of the training page. This will create an exact copy of the model for making changes.
    - For this next experiment, increase the learning rate to 0.008.
    - Number of epochs is increased to 7 or 8.
    - Click create to start a new model training process (DIGITS will allow the same model name).
    
<img src="images2/Digits_Screenshot8.png" alt="Drawing" style="width: 700px;"/>

- CONGRATULATIONS, you have trained your Convolutional Neural Network! DIGITS saves a copy of the network model at each epoch (it's one of the training parameters), so we can go back and analyze, or export, a model from any epoch of the training process.

<img src="images2/Digits_Screenshot9.png" alt="Drawing" style="width: 700px;"/>

### What can I do with this thing?
- Now, you can test it out on a couple training spectrograms, just for fun. The classification accuracy will be high, but this is just a quick sanity check.
- Go to the bottom of the model page and find 'Test a single image' and enter the path to the file '/data/traindemo/pos/posex_0.01db_2061.jpg'. Although it's not good practice to test with data from the training set, this let's us see the visualization features available in DIGITS.

<img src="images2/Digits_Screenshot10.png" alt="Drawing" style="width: 700px;"/>

- Make sure to check the 'Show visualizations and statistics' box. Then, click on the 'Classify One' button.
- The feed forward computation using our trained model is computed using the spectrogram we selected for input. The plots show the activations in each layer of the network.
- You should see the network classify correctly with a probability of approximately 99.98%.

<img src="images2/Screenshot13.png" alt="Drawing" style="width: 700px;"/>

# Neural Network Generalization on New Signals

### Using a test dataset 
A test data set is provided with 500-1000 positive and negative examples. The test set will be used to determine the generalization cability of the trained network. A range of SNR values are used in the test set in order to analyze the network's ability to discriminate low and high SNR signals. In doing so, you can come up with a probability of detection (PD) and a probability of false alarm (PFA) metrics for a subset of signal strengths. We wont be using DIGITS here! We will run a python script.

### Download saved network model
- In DIGITS, under the 'Trained Models' section, we can download the network weights and parameters for any epoch during training. If the model trained well, you would typically just download from the last epoch. For this tutorial, we have downloaded the model and prepared it for you. The path to the model is /data/model

<img src="images2/Screenshot_model.png" alt="Drawing" style="width: 600px;"/>

### Python analysis script
##### Note: We have already pre-installed the trained network model and test images.

- The file 'analyze_spect_dir.py' is provided. It has two functions, `detection_tst( pos_file_dir, model_dir, pic_type)` and `false_alarm_tst( neg_file_dir, model_dir, pic_type)`
- The functions take in arguments: positive/negative image file directory, caffe model directory, pic type ('jpg' or 'png')

In [ ]:
import analyze_spect_dir as ansp

- First, we will run the detection test, which runs all the positive class files through the caffe network model and tallies the detection rate (signal present and detected) and the miss rate (signal present but not detected).
- Note: Processing may take a few minutes, so be patient for the output to display.

In [ ]:
ansp.detection_tst('/data/testdemo/pos', '/data/model', 'jpg')

- After the above script has completed, you will see that the network has a relatively good detection probabilty, but most of the cases it cannot detect have low SNR < -5 dB.
- We will go back and fine tune the network to do better at lower SNR in the next section.
- Next, run the false alarm test script below. Here we input the negative test examples and see how many of them trigger a positive classification (i.e. a false detection).

In [ ]:
ansp.false_alarm_tst('/data/testdemo/neg', '/data/model', 'jpg')

- The probability-of-false-alarm (PFA) should be less than a few percent. 
- Note that this is one way to determine the classification strength of newly trained networks (of this type)!

### Fine tuning the network model
Now, let's look at a way to fine tune our model using an additional training dataset with very low SNRs, around -4 to -8 dB. Although we have provided this dataset for you, the steps you would take in DIGITS are listed here for educational purposes:
- Copy the training set over and make a new database in DIGITS for low SNR examples.
- Clone the trained classification model in DIGITS so we can train it again using the new dataset.
- In the 'Pretrained model' window, you can enter the snapshot .caffemodel that you saved off from the initial training. This serves as the initialization of the network which should already have good weights.
- Make the learning rate much smaller (/10 or /100) so the network cannot "unlearn" too much.
- Train on the low SNR data set for a handful of epochs. See if the network is able to improve.
- After training, you would import your new dataset and test it using ansp.detection_tst. For this tutorial we have done this for you.

In [ ]:
ansp.detection_tst('/data/testdemo/pos', '/data/model/finetunelow', 'jpg')

- With the provided tuned model, the network improves slightly on detecting low-SNR chirp signals while still detecting correctly on higher SNR examples.
- As we will see next, this secondary training with low-SNR also decreased the false-alarm rate (from 2.5% to 1.5).

In [ ]:
ansp.false_alarm_tst('/data/testdemo/neg', '/data/model/finetunelow', 'jpg')

- To further improve the network, we could
    - Train with more noise examples.
    - Freeze the CNN layer weights during this fine tuning so the network does not change too much.

### Post-Lab Summary

Following the steps outlined in this tutorial, you can create a detector for other signal types. For example, if you go to www.blog.kickview.com you can see an example using frequency hopped signals. 

This tutorial demonstrated signal detection using only two categories. It is not hard to extend this method to multiple categories by changing the model. We will demonstrate how to do this in the next tutorial. 

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well.